<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Fill-Tables-with-data-from-jsons" data-toc-modified-id="Fill-Tables-with-data-from-jsons-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Fill Tables with data from jsons</a></span><ul class="toc-item"><li><span><a href="#regattas" data-toc-modified-id="regattas-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>regattas</a></span></li><li><span><a href="#races" data-toc-modified-id="races-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>races</a></span></li><li><span><a href="#competitors" data-toc-modified-id="competitors-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>competitors</a></span></li><li><span><a href="#windsources-&amp;-wind" data-toc-modified-id="windsources-&amp;-wind-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>windsources &amp; wind</a></span></li><li><span><a href="#legs" data-toc-modified-id="legs-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>legs</a></span></li><li><span><a href="#race_comp" data-toc-modified-id="race_comp-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>race_comp</a></span></li><li><span><a href="#comp_leg" data-toc-modified-id="comp_leg-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>comp_leg</a></span></li><li><span><a href="#positions" data-toc-modified-id="positions-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>positions</a></span></li><li><span><a href="#courses" data-toc-modified-id="courses-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>courses</a></span></li><li><span><a href="#marks" data-toc-modified-id="marks-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>marks</a></span></li><li><span><a href="#marks_positions" data-toc-modified-id="marks_positions-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>marks_positions</a></span></li><li><span><a href="#Extra-functions-in-case-something-went-wrong-for-a-few-entries" data-toc-modified-id="Extra-functions-in-case-something-went-wrong-for-a-few-entries-1.12"><span class="toc-item-num">1.12&nbsp;&nbsp;</span>Extra functions in case something went wrong for a few entries</a></span></li></ul></li></ul></div>

# Fill Tables with data from jsons

In [8]:
###############################
## Tabellen vullen
###############################
import os, sys, inspect, json
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from lib.Utilities.globalVar import *
import lib.TransformData.combineData as combd
cd = combd.combineData()
import Download.download as download
from Utilities.converter import *

import pandas as pd
import numpy as np
from math import *
import urllib, json, requests
import pyodbc
from sqlalchemy import *
from sqlalchemy.exc import IntegrityError, InternalError
from sqlalchemy.types import BINARY, VARCHAR
from sqlalchemy import event
from sqlalchemy.types import TypeDecorator, CHAR
from sqlalchemy.dialects.postgresql import UUID
import uuid

In [9]:

server = 'sic-match-analysis.database.windows.net'
database = 'match-analysis'
username = 'sic-admin'
password = 'ZeilenIsLeuk!'
driver= '{ODBC Driver 17 for SQL Server}'
connection_str = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password


cnxn = pyodbc.connect(connection_str)
cursor = cnxn.cursor()

quoted = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))


@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True
        cursor.commit()

In [16]:
event  = [
        {'name' :  'hwcs2020-round1', 'regattaNameContaining': 'HWCS 2020 Round 1', 'raceNameContaining': ''}
        ,{'name' : 'www', 'regattaNameContaining': 'Enoshima', 'raceNameContaining': ''}
        ,{'name' :  'tokyo2019', 'regattaNameContaining': 'Tokyo', 'raceNameContaining': ''}
        ]
events = [{'name' :  'www', 'regattaNameContaining': 'WCS 2019 Genoa - 49er', 'raceNameContaining': ''},
         {'name' : 'www', 'regattaNameContaining': 'WCS 2019 Genoa - Nacra', 'raceNameContaining' : ''}
         ]
eventSelection = event

addNewRaces = True

## regattas

In [17]:
## TABLE 1 regattas
sql = """
SELECT TOP (0) * from regattas
"""
reg = pd.read_sql(sql, con = engine)

for serv in eventSelection:    
    urlb = urlbase.replace('www', serv['name'])
    url = urlb + 'regattas'
    data = getData(url)
    cd = combd.combineData(serv)
    df = cd.transformJsonToPD(data, 'regattas')
    reg = reg.append(df)
    toSql(df, 'regattas', engine)

reg = reg.reset_index(drop = True)

IntegrityError: Failed to upload regattas to sql :for regatta: 	, race: 	
IntegrityError: Failed to upload regattas to sql :for regatta: 	, race: 	
IntegrityError: Failed to upload regattas to sql :for regatta: 	, race: 	


In [18]:
reg


,regatta,boatclass,courseAreaId
0,HWCS 2020 Round 1 - Laser,Laser Int.,180a9561-4034-4cc6-aaa6-698338bfd5cc
1,HWCS 2020 Round 1 - Laser Radial,Laser Radial,180a9561-4034-4cc6-aaa6-698338bfd5cc
2,HWCS 2020 Round 1 - Finn,Finn,180a9561-4034-4cc6-aaa6-698338bfd5cc
3,HWCS 2020 Round 1 - Nacra 17,Nacra 17,180a9561-4034-4cc6-aaa6-698338bfd5cc
4,HWCS 2020 Round 1 - RS:X Men,RS:X,180a9561-4034-4cc6-aaa6-698338bfd5cc
5,HWCS 2020 Round 1 - RS:X Women,RS:X,180a9561-4034-4cc6-aaa6-698338bfd5cc
6,HWCS 2020 Round 1 - 470 Men,470,180a9561-4034-4cc6-aaa6-698338bfd5cc
7,HWCS 2020 Round 1 - 49er,49er,180a9561-4034-4cc6-aaa6-698338bfd5cc
8,HWCS 2020 Round 1 - 470 Women,470,180a9561-4034-4cc6-aaa6-698338bfd5cc
9,HWCS 2020 Round 1 - 49erFX,49erFX,180a9561-4034-4cc6-aaa6-698338bfd5cc


## races

In [13]:
# TABLE 2 races columns; 
# regatta
# race
# raceShort
# raceId
# maxWindSpd_kts
# minWindSpd_kts
# avgWindDir_deg
# firstlegbearing_deg

# coming from races, windsummary and firstlegbearing
filenames = ['races', 'windsummary', 'firstlegbearing']

for idx, row in reg.iterrows():
    if not addNewRaces:
        print("No new races are added, because you didn't want to")
        break
    regName = row.regatta
    
    url = urlbase + 'regattas/{}/races'.format(regName)
    data = getData(url)
    dic_data = {'regatta': regName, 'data': data}
    races = cd.transformJsonToPD(dic_data, 'races')
    
       
    url = urlbase + 'regattas/{}/windsummary'.format(regName)
    data = getData(url)
    dic_data = {'regatta': regName, 'data': data}
    windsummary = cd.transformJsonToPD(dic_data, 'windsummary').set_index(['regatta', 'raceShort', 'fleet'])   
    
    flb = []
    for index, race in races.iterrows():
        raceName = race.race               
        url = urlbase + 'regattas/{}/races/{}/firstlegbearing'.format(regName, raceName)
        data = getData(url)

        dic_data = {'race_id': race.race_id, 'data': data}
        firstlegbearing = cd.transformJsonToPD(dic_data, 'firstlegbearing') 
        flb.append(firstlegbearing)
    flb = pd.concat(flb).set_index('race_id')


    races = races.set_index(['regatta', 'raceShort', 'fleet'])
    result = pd.concat([races, windsummary], axis=1, join='outer').reset_index().set_index('race_id')
    result = pd.concat([result, flb], axis=1, join='outer').reset_index().rename(columns = {'index':'race_id'})

    toSql(result, 'races', engine, regName)



No new races are added, because you didn't want to


In [14]:
sql = """
SELECT top (0) race_id, regatta, race from races
"""
raceSelection = pd.read_sql(sql, con = engine)
for idx, row in reg.iterrows():
    sql = """
    SELECT race_id, regatta, race from races
    where regatta = '{}'
    """.format(row.regatta)
    raceSelection = raceSelection.append(pd.read_sql(sql, con = engine))
raceSelection = raceSelection.reset_index(drop = True)
print(raceSelection)
stop

                                            race_id  \
0             b'"M\xfap\xa4G\x017\x8e\xe4H\xf7PH#*'   
1       b'#\x1a\x01\xf0\xa4G\x017\x8f\x0eH\xf7PH#*'   
2             b'#\xf20\xb0\xa4G\x017\x8f8H\xf7PH#*'   
3          b'M\xb1\xeb@\xa2\xc8\x017`\x02H\xf7PH#*'   
4                b'Ng\xd9`\xa2\xc8\x017`.H\xf7PH#*'   
..                                              ...   
97      b'\xe3U\x9e\xc0\xa2\xc5\x017]\xe4H\xf7PH#*'   
98         b'\xef\xd2\x91 \xa5n\x017\xbczH\xf7PH#*'   
99      b'\xf0r\xbb\xd0\xa5n\x017\xbc\x92H\xf7PH#*'   
100     b'\xfb\x83r`\xa4\x9e\x017\xac\xbaH\xf7PH#*'   
101  b'\xfc\x11&\xc0\xa4\x9e\x017\xac\xd2H\xf7PH#*'   

                       regatta               race  
0    Tokyo 2019 - Laser Radial  R4 (Laser Radial)  
1    Tokyo 2019 - Laser Radial  R5 (Laser Radial)  
2    Tokyo 2019 - Laser Radial  R6 (Laser Radial)  
3    Tokyo 2019 - Laser Radial  R2 (Laser Radial)  
4    Tokyo 2019 - Laser Radial  R3 (Laser Radial)  
..                         

NameError: name 'stop' is not defined

In [ ]:
## add begin and end time to races
for idx, row in raceSelection.iterrows():
    
    regName = row.regatta
    raceName = row.race

    url = dataUrl(regName, raceName, 'times')
    data = getData(url, regName, raceName)
    dic_data = {'race_id': row.race_id, 'data': data}
    df = cd.transformJsonToPD(dic_data, 'times')#.set_index(['regatta', 'comp_name']) 
    times = [df['startOfRace-ms'].values[0],df['endOfRace-ms'].values[0]]
    if times[0] is None:
        print('No times found for {} {}'.format(regName, raceName))
        continue
    cursor.execute("""
        UPDATE races
        SET    races.[startOfRace-ms] = {}
               ,races.[endOfRace-ms] = {}
        WHERE
        regatta = '{}'
          and race = '{}'
          """.format(df['startOfRace-ms'].values[0],df['endOfRace-ms'].values[0] 
                     , regName , raceName)
                  )
    cursor.commit()

              


## competitors

In [ ]:
url = 'https://www.sapsailing.com/sailingserver/api/v1/regattas/datamining'
data = getData(url)
query_ids = pd.DataFrame(data)
query_ids.Identifier


In [ ]:
## TABLE 3 Competitors
for idx, row in reg.iterrows():
    regName = row.regatta
    
    url = urlbase + 'regattas/{}/entries'.format(regName)
    data = getData(url, regName)
    dic_data = {'regatta': regName, 'data': data}
    competitors = cd.transformJsonToPD(dic_data, 'entries').set_index(['regatta', 
                                                                      'comp_name'])

    dir_datamining =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) + \
                    webAddressToFilename(regattaLoc(regName)) + '/datamining/'
    datamining_reg = []

    for idx, q in query_ids.iterrows():
        url = urlbase + 'regattas/{}/datamining/{}'.format(regName, q.Identifier)
        data = getData(url)
        dic_data = {'regatta': regName, 'data': data}
        df = cd.transformJsonToPD(dic_data, q.Identifier).set_index(['regatta', 'comp_name']) 
        datamining_reg.append(df)

    datamining_reg = pd.concat(datamining_reg, axis = 1, join = 'outer')
    result = pd.merge(datamining_reg.reset_index(), competitors.reset_index(), how='left')
    
    toSql(result, 'competitors', engine, regName)

       

## windsources & wind

In [ ]:
## Table 4 wind

for indx, row in raceSelection.iterrows():    
    regName = row.regatta
    raceName = row.race

    ## quick check whether there are already windsources loaded
    print('Get first pos windsources', end = '\r')
    sql = """ SELECT * FROM {}
            WHERE race_id = {}
          """.format('windsources', idToText(row.race_id))
    check = pd.read_sql(sql, con = engine) ## already a positions for this race ?   
    if check.empty:
#                     print('\tAlready positions found in wind for \t{} \t{} \t{}'.format(regName, raceName, ws.typeName, ws.id))
        print('Fetch windsources data', end = '\r')        
        url_ws = windUrl(regName, raceName)
        data = requests.get(url_ws, timeout = 100)

        if data.status_code == 404:
            print('No windsources available for {} {}'.format(regName, raceName))
            continue
        data = data.json()
        print('Transform windsources data', end = '\r')     
        dic_data = {'race_id': row.race_id, 'data': data}
        df = cd.transformJsonToPD(data, 'windsources')
        df.insert(0,'race_id', row.race_id)
        f = toSql(df,'windsources', engine, regName, raceName)
    else:
        df = check
        print('already windsources found for {} {}'.format(regName, raceName))

    for idx, ws in df.iterrows():
        ## quick check whether there are already entries
        if ws.typeName != "EXPEDITION":
            continue
 
        print('Get first pos wind        ', end = '\r')
        sql = """ SELECT TOP (1) * FROM {}
                WHERE race_id = {}
                and windSource = 'EXPEDITION'
                and windSource_id = '{}'
              """.format('wind', idToText(row.race_id),  ws.id)
        check = pd.read_sql(sql, con = engine) ## already a positions for this race ?   
        if not check.empty:
            print('\tAlready positions found in wind for \t{} \t{} \t{}'.format(regName, raceName, ws.typeName, ws.id))
            continue
        print('load data from url', end = '\r')
        url_wind = windUrl(regName, raceName, windSource = ws.typeName, windSourceId = ws.id)
        data = requests.get(url_wind, timeout = 100)
        if data.status_code == 400:
            print('\tNo wind available for {} {} {}'.format(regName, raceName, ws.typeName))
            continue
        data = data.json()
        df = cd.transformJsonToPD(data, 'wind')
        if df.empty:
            print('\tNo wind available for {} {} {} {}'.format(regName, raceName, ws.typeName, ws.id))
            continue            
        df.windSource = df.windSource.apply(lambda x: ws.typeName)
        df.insert(0, 'windSource_id', ws.id)
        df.insert(0, 'race_id', row.race_id)
        print('Upload wind data to sql server', end = '\r')
        e = toSql(df, 'wind', engine, regName, raceName + ws.typeName + ws.id)



In [ ]:
check

## legs

In [ ]:
## TABLE 5 legs
# [regatta]
# ,[race]
# ,[leg_nr]
# ,[fromWaypointId]
# ,[toWaypointId]
# ,[to]
# ,[from]
# ,[upOrDownwindLeg]
# ,[leg_nr_from_finish]
# ,[distance]

           
for indx, row in raceSelection.iterrows():    
    regName = row.regatta
    raceName = row.race    
    url = dataUrl(regName, raceName, 'competitors/legs')
    data = getData(url)
    dic_data = {'regatta': regName, 'race_id': row.race_id, 'data': data}
    df = cd.transformJsonToPD(dic_data, 'legs') 
    
    e = toSql(df, 'legs', engine, regName, raceName)




## race_comp

In [ ]:
### TABLE 6
## This code create all entries in race_comp. We do this by checking that the nr_competitors in 
## the races table is still NULL because we did not update that yet.
## We will later add rank, based on comp_leg
sql = """
INSERT INTO race_comp(race_id, regatta, comp_id)
SELECT races.race_id, races.regatta, competitors.comp_id FROM races, competitors
WHERE races.regatta = competitors.regatta
 and races.nr_competitors IS NULL
"""
cursor.execute(sql)
cursor.commit()


## comp_leg

In [ ]:
## TABLE 7 comp_leg
# [regatta]
# ,[race]
# ,[leg_nr]
# ,[comp_id]
# ,[competitor_distanceTraveled-m]
# ,[competitor_averageSOG-kts]
# ,[competitor_tacks]
# ,[competitor_jibes]
# ,[competitor_penaltyCircles]
# ,[competitor_rank]
# ,[competitor_gapToLeader-s]
# ,[competitor_gapToLeader-m]
# ,[competitor_started]
# ,[competitor_finished]

for indx, row in raceSelection.iterrows():    
    regName = row.regatta
    raceName = row.race    
    url = dataUrl(regName, raceName, 'competitors/legs')
    data = getData(url)
    dic_data = {'regatta': regName, 'race_id': row.race_id, 'data': data}
    df = cd.transformJsonToPD(dic_data, 'comp_leg')     
    e = toSql(df, 'comp_leg', engine, regName, raceName)




## positions

In [ ]:
### Markpassings, to update legnr_columnn in positions

cursor.execute("""
DROP TABLE temp_markpassings
CREATE TABLE temp_markpassings
(race_id binary (16) NOT NULL,
comp_id binary(16) NOT NULL,
leg_nr int NOT NULL,
begin_leg_ms bigint NULL,
end_leg_ms bigint NULL

)

CREATE UNIQUE INDEX idx ON temp_markpassings (race_id, leg_nr, comp_id);
CREATE CLUSTERED INDEX idx_2 ON temp_markpassings (race_id, comp_id);

""")

cursor.execute("""
ALTER TABLE temp_markpassings
ADD CONSTRAINT fk_temp_markpassings2_comp_leg FOREIGN KEY (race_id, leg_nr, comp_id)
      REFERENCES comp_leg (race_id, leg_nr, comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")
cursor.commit()

In [ ]:
#### TEMPORARY TABLE MARKPASSINGS

for indx, row in raceSelection.iterrows():    
    regName = row.regatta
    raceName = row.race    
    url = dataUrl(regName, raceName, 'markpassings')
    data = getData(url)
    if data is None:
        continue
    dic_data = {'regatta': regName, 'race_id': row.race_id, 'data': data}
    df = cd.transformJsonToPD(dic_data, 'markpassings')     
    e = toSql(df, 'temp_markpassings', engine, regName, raceName)
    


In [ ]:
data = requests.get(url, timeout = 100)
data.status_code == 404

In [ ]:
## TABLE 8 positions

for ids, row in raceSelection.iterrows():
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    
    print('Get first positions\t\t\t', end = '\r')
    sql = """ SELECT * FROM {}
            WHERE leg_nr = 1 and race_id = {}
          """.format('positions', race_id)
    check = pd.read_sql(sql, con = engine) ## already a positions for this race ?   
    if not check.empty:
        print('Already positions found in db for \t{} \t{}'.format(regName, raceName))
        continue

    print('get markpassings\t\t\t', end="\r")
    sql = """ SELECT * FROM temp_markpassings
            WHERE race_id = {}
            """.format(race_id)
    mps = pd.read_sql(sql, con = engine) ## markpassings for this race
    if mps.empty:
        print('No markpassings where found for\t{} \t{}'.format(regName, raceName))
        continue        
        
    print('Fetch positions data\t\t\t', end = '\r')        
    url_ws = dataUrl(regName, raceName, 'positions_comp')
    data = requests.get(url_ws, timeout = 100)

    if data.status_code == 404:
        print('No positions available for {} {}'.format(regName, raceName))
        continue
    data = data.json()
    
    dic_data = {'regatta': regName, 'race_id': row.race_id, 'data': data}
    print('Transform positions data\t\t\t', end = '\r')        
    df = cd.transformJsonToPD(dic_data, 'competitor_positions')

    print('determine leg numbers\t\t\t', end="\r")
    for idx, comp_leg in mps.iterrows():
        df.loc[(df.comp_id == comp_leg.comp_id) & (df['timepoint_ms'] > comp_leg.begin_leg_ms)
               & (df['timepoint_ms'] < comp_leg.end_leg_ms),'leg_nr'] = comp_leg.leg_nr
    df = df.dropna(subset=['leg_nr'])
    
    toSql(df,'positions', engine, regName, raceName)
    
    
    



## courses

In [ ]:
## Table 9 couses
cd.uploadRaceData(raceSelection, 'course', 'course', 'courses', engine )

## marks

In [ ]:
## Table 10 marks
cd.uploadRaceData(raceSelection, 'marks/positions', 'marks', 'marks', engine )

## marks_positions

In [ ]:
## Table 9 marks_positions
cd.uploadRaceData(raceSelection, 'marks/positions', 'marks_positions', 'marks_positions', engine )

## Extra functions in case something went wrong for a few entries

In [ ]:
### Extra functions to manually add tracks of seperate competitors
## in the previous cell tracks have been added per race, but for some reason it went wrong in some cases

def uploadPositions(regName, race, comp_id):
    dir_races =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) + \
                    webAddressToFilename(regattaLoc(regName)) + '/races/'
    filepath = dir_races + webAddressToFilename(race) + "/competitors/positions.json"
    sql = """ SELECT * FROM temp_markpassings
            WHERE regatta = '{}'
            and race = '{}'
            and comp_id = {}
            """.format(regName, race, comp_id)
    mps = pd.read_sql(sql, con = engine) ## markpassings for this race
    
    if mps.empty:
        print('No markpassings found, try to download them again')
    
    with open(filepath) as json_file:
#                     print('file extracted: ', filepath)
        data = json.load(json_file)
    dic_data = {'regatta': regName, 'race': filenameToWebAddress(race), 'data': data}

    print('transform to pd', end="\r")
    df = cd.transformJsonToPD(dic_data, 'competitor_positions') 

    print('determine leg numbers', end="\r")
    for idx, comp_leg in mps.iterrows():
        df.loc[(df.comp_id == comp_leg.comp_id) & (df['timepoint_ms'] > comp_leg.begin_leg_ms)
               & (df['timepoint_ms'] < comp_leg.end_leg_ms),'leg_nr'] = comp_leg.leg_nr
    df = df.dropna(subset=['leg_nr'])
#                 print(mps.head())
#                 print(df.head())

    try:
        print('Uploading positions to db', end="\r")
        df.to_sql('positions', con = engine, index = False, if_exists = 'append', chunksize = 10000)
        print('Added positions of \t{} \t{} \t succesfully to the database'.format(regName, race))
    except IntegrityError as e:
        print('Could not add positions of \t{} \t{} \tto the database because of IntegrityError.'.format(regName,race))
    return df
